In [66]:
import pandas as pd
from openpyxl import workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import numpy as np
import os

### Loading file to dataframes

Loading .xlsx file created in agesex.ipynb

In [67]:
# Creating ordered list of DFs to name each sheet coming in from workbook
df_names = ['DF_total_all','DF_male_all','DF_female_all','DF_total_whi',
            'DF_total_male_whi','DF_total_female_whi','DF_total_baa',
            'DF_total_male_baa','DF_total_female_baa','DF_total_aian',
            'DF_total_male_aian','DF_total_female_aian','DF_total_aa',
            'DF_total_male_aa','DF_total_female_aa','DF_total_nhop',
            'DF_total_male_nhop','DF_total_female_nhop','DF_total_sor',
            'DF_total_male_sor','DF_total_female_sor','DF_total_tom',
            'DF_total_male_tom','DF_total_female_tom','DF_total_hol',
            'DF_total_male_hol','DF_total_female_hol'
            ]

In [68]:
# Dictionary to store the dataframes
dfs = {}

# Get the path of the current working directory
cwd = os.getcwd()

# Construct the full path to the .xlsx file
file_path = os.path.join(cwd, '2020_agesex_data.xlsx')

# Load each sheet of the .xlsx file into a named dataframe
for name in df_names:
    df = pd.read_excel(file_path, sheet_name=name, header=None)  # Specify header=None to treat the first row as data
    df.columns = df.iloc[0]  # Set the first row as the column headers
    df = df[1:]  # Exclude the first row from the data
    df.reset_index(drop=True, inplace=True)  # Reset the index
    dfs[name] = df

### Verifying dataframes load in

Print all the names of the dataframes loaded in

In [69]:
# Print the names of the dataframes in 'dfs'
print("Data Frames in 'dfs':")
for name in dfs.keys():
    print(name)

Data Frames in 'dfs':
DF_total_all
DF_male_all
DF_female_all
DF_total_whi
DF_total_male_whi
DF_total_female_whi
DF_total_baa
DF_total_male_baa
DF_total_female_baa
DF_total_aian
DF_total_male_aian
DF_total_female_aian
DF_total_aa
DF_total_male_aa
DF_total_female_aa
DF_total_nhop
DF_total_male_nhop
DF_total_female_nhop
DF_total_sor
DF_total_male_sor
DF_total_female_sor
DF_total_tom
DF_total_male_tom
DF_total_female_tom
DF_total_hol
DF_total_male_hol
DF_total_female_hol


Print all the name and shape of the dataframes loaded in

In [70]:
# Print the shape of the dataframes in 'dfs'
print("Data Frames in 'dfs':")
for name in dfs.keys():
    print(name)
    print(dfs[name].shape)

Data Frames in 'dfs':
DF_total_all
(3221, 5)
DF_male_all
(3221, 108)
DF_female_all
(3221, 108)
DF_total_whi
(3221, 5)
DF_total_male_whi
(3221, 108)
DF_total_female_whi
(3221, 108)
DF_total_baa
(3221, 5)
DF_total_male_baa
(3221, 108)
DF_total_female_baa
(3221, 108)
DF_total_aian
(3221, 5)
DF_total_male_aian
(3221, 108)
DF_total_female_aian
(3221, 108)
DF_total_aa
(3221, 5)
DF_total_male_aa
(3221, 108)
DF_total_female_aa
(3221, 108)
DF_total_nhop
(3221, 5)
DF_total_male_nhop
(3221, 108)
DF_total_female_nhop
(3221, 108)
DF_total_sor
(3221, 5)
DF_total_male_sor
(3221, 108)
DF_total_female_sor
(3221, 108)
DF_total_tom
(3221, 5)
DF_total_male_tom
(3221, 108)
DF_total_female_tom
(3221, 108)
DF_total_hol
(3221, 5)
DF_total_male_hol
(3221, 108)
DF_total_female_hol
(3221, 108)


Print all the names & dtype of the dataframes loaded in

In [71]:
# Print the names & dtype of the dataframes in 'dfs'
print("Data Frames in 'dfs':")
for name in dfs.keys():
    print(name)
    print(dfs[name].dtypes)

Data Frames in 'dfs':
DF_total_all
0
Total       object
Location    object
State       object
County      object
FIPS        object
dtype: object
DF_male_all
0
Total                  object
Under 1 Year           object
1 Year                 object
2 Years                object
3 Years                object
                        ...  
110  Years and Over    object
Location               object
State                  object
County                 object
FIPS                   object
Length: 108, dtype: object
DF_female_all
0
Total                  object
Under 1 Year           object
1 Year                 object
2 Years                object
3 Years                object
                        ...  
110  Years and Over    object
Location               object
State                  object
County                 object
FIPS                   object
Length: 108, dtype: object
DF_total_whi
0
Total       object
Location    object
State       object
County      object
FIPS        object


Print all the column names of the dataframes loaded in

In [72]:
# Iterate over the dataframes in 'dfs'
for key, df in dfs.items():
    print(f"Columns of dataframe '{key}': {list(df.columns)}")

Columns of dataframe 'DF_total_all': ['Total', 'Location', 'State', 'County', 'FIPS']
Columns of dataframe 'DF_male_all': ['Total', 'Under 1 Year', '1 Year', '2 Years', '3 Years', '4 Years', '5 Years', '6 Years', '7 Years', '8 Years', '9 Years', '10 Years', '11 Years', '12 Years', '13 Years', '14 Years', '15 Years', '16 Years', '17 Years', '18 Years', '19 Years', '20 Years', '21 Years', '22 Years', '23 Years', '24 Years', '25 Years', '26 Years', '27 Years', '28 Years', '29 Years', '30 Years', '31 Years', '32 Years', '33 Years', '34 Years', '35 Years', '36 Years', '37 Years', '38 Years', '39 Years', '40 Years', '41 Years', '42 Years', '43 Years', '44 Years', '45 Years', '46 Years', '47 Years', '48 Years', '49 Years', '50 Years', '51 Years', '52 Years', '53 Years', '54 Years', '55 Years', '56 Years', '57 Years', '58 Years', '59 Years', '60 Years', '61 Years', '62 Years', '63 Years', '64 Years', '65 Years', '66 Years', '67 Years', '68 Years', '69 Years', '70 Years', '71 Years', '72 Years'

### Building weighted average column

Grouping Dataframes with like structures

In [73]:
stats_df = ['DF_male_all','DF_female_all','DF_total_male_whi',
             'DF_total_female_whi','DF_total_male_baa','DF_total_female_baa',
             'DF_total_male_aian','DF_total_female_aian','DF_total_male_aa',
             'DF_total_female_aa','DF_total_male_nhop','DF_total_female_nhop',
             'DF_total_male_sor','DF_total_female_sor','DF_total_male_tom',
             'DF_total_female_tom','DF_total_male_hol','DF_total_female_hol'
             ]

Setting new column names and columns to be typed to integer

In [74]:
col_names = ['Total','0','1','2','3','4','5','6','7','8','9','10','11','12',
             '13','14','15','16','17','18','19','20','21','22','23','24','25',
             '26','27','28','29','30','31','32','33','34','35','36','37','38',
             '39','40','41','42','43','44','45','46','47','48','49','50','51',
             '52','53','54','55','56','57','58','59','60','61','62','63','64',
             '65','66','67','68','69','70','71','72','73','74','75','76','77',
             '78','79','80','81','82','83','84','85','86','87','88','89','90',
             '91','92','93','94','95','96','97','98','99','102','107','110',
             'Location','State','County','FIPS'
             ]

col_ints = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14',
            '15','16','17','18','19','20','21','22','23','24','25','26','27',
            '28','29','30','31','32','33','34','35','36','37','38','39','40',
            '41','42','43','44','45','46','47','48','49','50','51','52','53',
            '54','55','56','57','58','59','60','61','62','63','64','65','66',
            '67','68','69','70','71','72','73','74','75','76','77','78','79',
            '80','81','82','83','84','85','86','87','88','89','90','91','92',
            '93','94','95','96','97','98','99','102','107','110'
            ]

In [75]:
# Iterate over the dataframes in 'dfs'
for key, df in dfs.items():
    # Check if the dataframe is present in 'stats_df'
    if key in stats_df:
        # Print the type of each column name as a list
        column_name_types = [f"{col}: {type(col).__name__}" for col in df.columns]
        print(f"Column name types of dataframe '{key}':")
        print(column_name_types)
        print()

Column name types of dataframe 'DF_male_all':
['Total: str', 'Under 1 Year: str', '1 Year: str', '2 Years: str', '3 Years: str', '4 Years: str', '5 Years: str', '6 Years: str', '7 Years: str', '8 Years: str', '9 Years: str', '10 Years: str', '11 Years: str', '12 Years: str', '13 Years: str', '14 Years: str', '15 Years: str', '16 Years: str', '17 Years: str', '18 Years: str', '19 Years: str', '20 Years: str', '21 Years: str', '22 Years: str', '23 Years: str', '24 Years: str', '25 Years: str', '26 Years: str', '27 Years: str', '28 Years: str', '29 Years: str', '30 Years: str', '31 Years: str', '32 Years: str', '33 Years: str', '34 Years: str', '35 Years: str', '36 Years: str', '37 Years: str', '38 Years: str', '39 Years: str', '40 Years: str', '41 Years: str', '42 Years: str', '43 Years: str', '44 Years: str', '45 Years: str', '46 Years: str', '47 Years: str', '48 Years: str', '49 Years: str', '50 Years: str', '51 Years: str', '52 Years: str', '53 Years: str', '54 Years: str', '55 Years:

Iterating through grouped dataframes to change column name and type

In [76]:
# Iterate over the dataframes in 'dfs'
for key, df in dfs.items():
    # Check if the dataframe is present in 'stats_df'
    if key in stats_df:
        # Modify column names based on 'col_names' list
        df.columns = col_names[:len(df.columns)]

        # Loop through the columns to convert to integers
        for col in df.columns:
            if col in col_ints:
                # Change the column name to an integer
                df.rename(columns={col: int(col)}, inplace=True)
                # Convert the column values to integers
                df[int(col)] = df[int(col)].astype(int)

Print all the names & dtype of the modified dataframes

In [77]:
# Iterate over the dataframes in 'dfs'
for key, df in dfs.items():
    # Check if the dataframe is present in 'stats_df'
    if key in stats_df:
        # Print the type of each column name as a list
        column_name_types = [f"{col}: {type(col).__name__}" for col in df.columns]
        print(f"Column name types of dataframe '{key}':")
        print(column_name_types)
        print()

Column name types of dataframe 'DF_male_all':
['Total: str', '0: int', '1: int', '2: int', '3: int', '4: int', '5: int', '6: int', '7: int', '8: int', '9: int', '10: int', '11: int', '12: int', '13: int', '14: int', '15: int', '16: int', '17: int', '18: int', '19: int', '20: int', '21: int', '22: int', '23: int', '24: int', '25: int', '26: int', '27: int', '28: int', '29: int', '30: int', '31: int', '32: int', '33: int', '34: int', '35: int', '36: int', '37: int', '38: int', '39: int', '40: int', '41: int', '42: int', '43: int', '44: int', '45: int', '46: int', '47: int', '48: int', '49: int', '50: int', '51: int', '52: int', '53: int', '54: int', '55: int', '56: int', '57: int', '58: int', '59: int', '60: int', '61: int', '62: int', '63: int', '64: int', '65: int', '66: int', '67: int', '68: int', '69: int', '70: int', '71: int', '72: int', '73: int', '74: int', '75: int', '76: int', '77: int', '78: int', '79: int', '80: int', '81: int', '82: int', '83: int', '84: int', '85: int', '86

In [78]:
# Iterate over the dataframes in 'dfs'
for key, df in dfs.items():
    # Set the 'name' attribute of the DataFrame to the key
    df.name = key

In [79]:
# Calculate the weighted average for each dataframe
for key, df in dfs.items():
    # Check if the dataframe exists in the stats_df list
    if key in stats_df:
        # Calculate the weighted average
        weighted_avg = np.dot(df[col_ints], df[col_ints].values)
        # Add the weighted average as a new column 'Average Age'
        df['Average Age'] = weighted_avg.round().astype(int)
# Load the dataframes back into the dictionary
dfs = {key: df for key, df in dfs.items()}
# Print the dataframes
for key, df in dfs.items():
    print(df)

KeyError: "None of [Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',\n       ...\n       '93', '94', '95', '96', '97', '98', '99', '102', '107', '110'],\n      dtype='object', length=103)] are in the [columns]"

In [ ]:
print(dfs['DF_male_all'])

      Total    0    1    2    3    4    5    6    7    8  ...  97  98  99  \
0     40783  437  450  511  484  513  525  576  559  549  ...   3   1   0   
1      6205   70   70   84   91   80   90  116   67   97  ...   0   0   0   
2      6136   72   80   83   60   82   78   69   84   67  ...   3   1   0   
3     17906  157  203  182  171  192  161  202  186  158  ...   4   0   0   
4      5722   64   51   65   60   62   79   85   71   76  ...   0   0   0   
...     ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ..  ..  ..   
3216   8345   98   86  120  100  109  120  115   96  114  ...   0   3   1   
3217  13569  183  124  170  184  175  185  185  160  188  ...   1   1   0   
3218  19687  190  218  223  195  287  247  211  223  237  ...   6   4   2   
3219   1684   20   10    4   14    9   10   16   17   18  ...   0   0   0   
3220  49310  593  582  669  639  691  704  658  650  687  ...   9   1   0   

      102  107  110                     Location  State  County   FIPS  
0 

In [ ]:
# Iterate over the dataframes in 'dfs'
for key, df in dfs.items():
    # Check if the dataframe is present in 'stats_df'
    if key in stats_df:
        # Print the type of each column name as a list
        column_name_types = [f"{col}: {type(col).__name__}" for col in df.columns]
        print(f"Column name types of dataframe '{key}':")
        print(column_name_types)
        print()

Column name types of dataframe 'DF_male_all':
['Total: str', '0: int', '1: int', '2: int', '3: int', '4: int', '5: int', '6: int', '7: int', '8: int', '9: int', '10: int', '11: int', '12: int', '13: int', '14: int', '15: int', '16: int', '17: int', '18: int', '19: int', '20: int', '21: int', '22: int', '23: int', '24: int', '25: int', '26: int', '27: int', '28: int', '29: int', '30: int', '31: int', '32: int', '33: int', '34: int', '35: int', '36: int', '37: int', '38: int', '39: int', '40: int', '41: int', '42: int', '43: int', '44: int', '45: int', '46: int', '47: int', '48: int', '49: int', '50: int', '51: int', '52: int', '53: int', '54: int', '55: int', '56: int', '57: int', '58: int', '59: int', '60: int', '61: int', '62: int', '63: int', '64: int', '65: int', '66: int', '67: int', '68: int', '69: int', '70: int', '71: int', '72: int', '73: int', '74: int', '75: int', '76: int', '77: int', '78: int', '79: int', '80: int', '81: int', '82: int', '83: int', '84: int', '85: int', '86